In [16]:
import numpy as np
import scipy as sp
import matplotlib
from matplotlib import pyplot as plt
import pytraj as pt
import pandas as pd
import sys
import collections
import gc
import copy

In [85]:
#import calcpka data for equil run

runDir='md_steps'
steps=['equil']
systems=['WT']

nWindows=1 #4 #only window_00 is done so far
windowStr='window_%02g'
windows=[windowStr%iWin for iWin in np.arange(nWindows)]
windowStart_pH=6.0
windowDelta_pH=1.0

popFileExt='populations.dat'
popFrames=[]
popCols=['Unused1','Resname','Resnum',
               'Pop_0','Crg_0','Pop_1','Crg_1',
               'Pop_2','Crg_2','Pop_3','Crg_3']
popIDcols=['System','Window','pH','Resname','Resnum']
popSkipRows=2

for system in systems:
    sysDir='/'.join([runDir,system])
    for step in steps:
        stepDir='/'.join([sysDir,step])
        for iWin,window in enumerate(windows):
            window_pH=windowStart_pH+iWin*windowDelta_pH
            popFileName='.'.join([system,step,window,popFileExt])
            popFilePath='/'.join([stepDir,popFileName])
            print 'Loading %s'%popFileName
            sys.stdout.flush()
            testFrame=pd.read_csv(popFilePath,
                                  delim_whitespace=True,
                                  skiprows=popSkipRows,
                                  names=popCols)
            testFrame=testFrame.drop(columns=[
                dCol for dCol in testFrame.columns if 'Unused' in dCol])
            testFrame['System']=system
            testFrame['Window']=window
            testFrame['pH']=window_pH
            testFrameLong=pd.melt(
                frame=testFrame,id_vars=popIDcols)
            popFrames.append(copy.deepcopy(testFrameLong))
            gc.collect()
popDataTable=pd.concat(popFrames)
popFrames=[]
testFrame=[]
testFrameLong=[]
popDataTable.head()

Loading WT.equil.window_00.populations.dat


,System,Window,pH,Resname,Resnum,variable,value
0,WT,window_00,6.0,HIP,15,Pop_0,0.974162
1,WT,window_00,6.0,CYS,17,Pop_0,1
2,WT,window_00,6.0,CYS,42,Pop_0,1
3,WT,window_00,6.0,LYS,45,Pop_0,1
4,WT,window_00,6.0,LYS,64,Pop_0,1


In [87]:
popNAdrop=popDataTable.dropna()
popSubDat=popNAdrop[popNAdrop.variable.apply(lambda x: 'Pop' in x)]
popSubDat['State']=0
popSubDat.State=popSubDat.variable.apply(lambda x: x.split('_')[1])
popSubDat['Population']=popSubDat.value
popSubDat=popSubDat[
    np.concatenate([popIDcols,['State','Population']])]

crgSubDat=popNAdrop[popNAdrop.variable.apply(lambda x: 'Crg' in x)]
crgSubDat.value=crgSubDat.value.map(lambda x: x.replace('(','').replace(')',''))
crgSubDat['State']=0
crgSubDat.State=crgSubDat.variable.apply(lambda x: x.split('_')[1])
crgSubDat['N_Prot']=crgSubDat.value
crgSubDat=crgSubDat[
    np.concatenate([popIDcols,['State','N_Prot']])]

print popSubDat.head()
print crgSubDat.head()

popSubDat=popSubDat.set_index(list(np.concatenate([popIDcols,['State']])))
crgSubDat=crgSubDat.set_index(list(np.concatenate([popIDcols,['State']])))
popCleaned=popSubDat.join(how='outer',
    other=crgSubDat)
popCleaned=popCleaned.reset_index()

popCleaned.to_csv('WT.joint.populations.csv',index=False)

popCleaned.head()


  System     Window   pH Resname  Resnum State Population
0     WT  window_00  6.0     HIP      15     0   0.974162
1     WT  window_00  6.0     CYS      17     0          1
2     WT  window_00  6.0     CYS      42     0          1
3     WT  window_00  6.0     LYS      45     0          1
4     WT  window_00  6.0     LYS      64     0          1
   System     Window   pH Resname  Resnum State N_Prot
46     WT  window_00  6.0     HIP      15     0      2
47     WT  window_00  6.0     CYS      17     0      1
48     WT  window_00  6.0     CYS      42     0      1
49     WT  window_00  6.0     LYS      45     0      3
50     WT  window_00  6.0     LYS      64     0      3


/Users/wbotellosmith/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/wbotellosmith/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/wbotellosmith/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,System,Window,pH,Resname,Resnum,State,Population,N_Prot
0,WT,window_00,6.0,HIP,15,0,0.974162,2
1,WT,window_00,6.0,CYS,17,0,1,1
2,WT,window_00,6.0,CYS,42,0,1,1
3,WT,window_00,6.0,LYS,45,0,1,3
4,WT,window_00,6.0,LYS,64,0,1,3
